## Notebook setup

In [ ]:
# Standard libraries
import os
import sys

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

from jupyterthemes import jtplot

# Numerical libraries
import numpy as np
import pandas as pd
import patsy as pt
import pymc3 as pm
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.formula.api import ols
from theano import tensor as T

# Internal libraries
sys.path.append('../../src')

# import lib.reconstruction.errors as errs
from lib.reconstruction.errors import get_errors_per_location
from lib.reconstruction.neighbors import FilterByOccupied
from lib.reconstruction.neighbors import get_adjacency, get_adjacency_per_location

from lib.reconstruction.bayes.poisson import build_poisson_model

In [ ]:
# Notebook configuration
pd.set_option('display.max_columns', 40)
# os.environ['THEANO_FLAGS'] = 'device=cuda,floatX=float32'

sns.set_style('white')
sns.set_context('paper')

colors = sns.cubehelix_palette(n_colors=2, start=0.5, hue=1, rot=.1, light=.65) 
colors += sns.cubehelix_palette(n_colors=2, start=2.5, hue=1, rot=.1, light=.65)

%matplotlib inline

## Load data

In [ ]:
tidy = pd.read_csv('../../etc/reconstruction/tidy_data.csv', index_col=0)

tidy['Condition'] = tidy['Condition'].map(lambda x: 'Untrained' if x == 'Naive' else x)
tidy['Position ID'] = tidy['Position ID'].map(int)

vals = ['Black Position', 'White Position', 
        'Is Real', 'Num Pieces']

board_set = tidy.pivot_table(index='Position ID', 
                             values=vals, 
                             aggfunc=lambda x: x.unique()[0])[vals]

### Data preprocessing

In [ ]:
# Compute the adjacency of each location

adjacencies = board_set.apply(get_adjacency_per_location, axis=1)

adjacency_column_names = ['adjacency_all', 
                          'adjacency_same', 
                          'adjacency_opposite']

adjacency_df = pd.DataFrame(adjacencies.tolist(), 
                            index=board_set.index, 
                            columns=adjacency_column_names)

In [ ]:
def get_occupied_mask(row):
    """Return indicators for whether a piece was at a location"""
    bp = np.stack([int(i) for i in row['Black Position']])
    wp = np.stack([int(i) for i in row['White Position']])
    p = bp + wp
    return p.tolist()

def get_condition_mask(condition):
    """Return convenience indicators for condition"""
    return [condition, ] * 36

for error_type in range(1, 4):
    tidy[f'errors_{error_type}'] = tidy.apply(
        lambda x: get_errors_per_location(x, str(error_type)), axis=1)

# Compute occupancy and condition indicators 
# for each location on each trial
tidy['occupied'] = tidy.apply(get_occupied_mask, axis=1)
tidy['condition_mask'] = tidy['Condition'].map(get_condition_mask)

# For convenience, 
# pull per-location adjacency statistics for each trial
# into the per-trial dataframe
tidy['adjacency_same'] = tidy['Position ID'].map(adjacency_df['adjacency_same'])
tidy['adjacency_opposite'] = tidy['Position ID'].map(adjacency_df['adjacency_opposite'])

# Convert the subject UID to an integer index
subject_ids = tidy['Subject ID'].unique()
subject_index = np.arange(len(subject_ids))
subject_index_map = dict(zip(subject_ids, subject_index))

tidy['subject_idx'] = tidy['Subject ID'].map(subject_index_map)

In [ ]:
def get_error_rates(df):
    """(DEPRECATED?) Return type 2 average error rates in a vector"""
    return np.stack(df['errors_2'], axis=1).mean(axis=1)

# Per-position mean error rates (over subjects)
g = tidy.groupby('Position ID')
board_set['errors'] = g.apply(get_error_rates)

# Get a dummy array of location indices for convenience
board_set['location_idx'] = np.tile(np.arange(36, dtype=np.uint8), [len(board_set), 1]).tolist()

# Get distances to center as a dummy field
# (same for all positions!)
blank_board = np.zeros((4, 9))
center = (blank_board.shape[0] / 2 - .5, blank_board.shape[1] / 2 - .5)

distances = np.sqrt(((np.argwhere(blank_board == 0) - center) ** 2).sum(axis=1))
board_set['distance_to_center'] = np.tile(distances, [len(board_set), 1]).tolist()

In [ ]:
# Construct filter functions
occupied_error_filter = FilterByOccupied('errors')
occupied_same_filter = FilterByOccupied('adjacency_same')
occupied_opposite_filter = FilterByOccupied('adjacency_opposite')
occupied_distances_filter = FilterByOccupied('distance_to_center')

# Create a dataframe with both board set metadata and adjacency measures
sum_df = pd.concat([board_set, adjacency_df], axis=1)

# Apparently there were some null outputs here? 
# Don't recall why...
sum_df = sum_df.loc[pd.notnull(sum_df['errors'])]

# For each board, filter the measurements to discard unoccupied locations
sum_df['occupied_errors'] = sum_df.apply(occupied_error_filter, axis=1)
sum_df['occupied_adjacency_same'] = sum_df.apply(occupied_same_filter, axis=1)
sum_df['occupied_adjacency_opposite'] = sum_df.apply(occupied_opposite_filter, axis=1)
sum_df['occupied_distance_to_center'] = sum_df.apply(occupied_distances_filter, axis=1)

In [ ]:
# Set up data for use with PyMC3
# New dataframe should have *locations* in individual rows

def expand_indicators(x):
    return np.stack([int(x), ] * 36)

# Extract exogenous variables
x_same = np.concatenate(tidy['adjacency_same'].values)
x_opposite = np.concatenate(tidy['adjacency_opposite'].values)
x_occupied = np.concatenate(tidy['occupied'].values)
x_condition_mask = np.concatenate(tidy['condition_mask'].values)

x_subject = np.concatenate(tidy['subject_idx'].map(expand_indicators).values)
x_position_type = np.concatenate(tidy['Is Real'].map(expand_indicators).values)
x_position_id = np.concatenate(tidy['Position ID'].map(expand_indicators).values)

# Extract endogenous variables
y1 = np.concatenate(tidy['errors_1'].values)
y2 = np.concatenate(tidy['errors_2'].values)
y3 = np.concatenate(tidy['errors_3'].values)

columns = ['subject', 'condition_mask', 'occupied', 
           'same', 'opposite', 
           'position_type', 'position_id',
           'errors_1', 'errors_2', 'errors_3']
                               
df_data = np.stack((x_subject, x_condition_mask, x_occupied, 
                    x_same, x_opposite, 
                    x_position_type, x_position_id,
                    y1, y2, y3)).T

bayes_model_df = pd.DataFrame(data=df_data, columns=columns)

# Force pandas to use integers
# Why doesn't it by default? Who knows, but it's dumb.
for c in bayes_model_df.columns:
    if c not in ['condition_mask', 'same', 'opposite']:
        bayes_model_df[c] = bayes_model_df[c].astype(int)

bayes_model_df['condition_indicator'] = bayes_model_df['condition_mask'].map(
    {'Trained': 1, 'Untrained': 0}).astype(int)

In [ ]:
# Set up condition and position type indicators
trained_sel = bayes_model_df['condition_mask'] == 'Trained'
untrained_sel = bayes_model_df['condition_mask'] == 'Untrained'
natural_sel  = bayes_model_df['position_type'] == 1
synthetic_sel = bayes_model_df['position_type'] == 0

In [ ]:
# Convert subject indices for columnar format
# Eg, indexes start from 0 for both trained/untrained groups

bayes_model_df.loc[untrained_sel, 'subject'] = bayes_model_df.loc[untrained_sel, 'subject'] % 19

# But still cache a unique indicator for each subject
bayes_model_df['usubject'] = bayes_model_df['subject'] 
bayes_model_df['usubject'] += bayes_model_df['condition_indicator'] * 19

# Same for real/fake positions
natural_ids = bayes_model_df.loc[natural_sel, 'position_id'].astype(int)
bayes_model_df.loc[natural_sel, 'position_id'] = natural_ids - natural_ids.min()

# Get an integer indicator for the *type* of error that was made
# Where a 0 indicates no error
error_columns = ['errors_1', 'errors_2', 'errors_3']
bayes_model_df['has_error'] = bayes_model_df[error_columns].astype(int).sum(axis=1)

error_sel = bayes_model_df['has_error'] == 1

error_values = bayes_model_df.loc[error_sel, error_columns].values
error_types = np.argmax(error_values, axis=1) + 1

bayes_model_df['error_type'] = 0
bayes_model_df.loc[error_sel, 'error_type'] = error_types

In [ ]:
# Create a pivot table, per subject, for locations with errors
has_error_df = bayes_model_df.loc[error_sel]
bayes_subject_piv = has_error_df.pivot_table(index='usubject', 
                                             columns='error_type',
                                             values='position_id',
                                             aggfunc=len)

# Total counts
bayes_subject_piv['n'] = bayes_subject_piv[[1, 2, 3]].sum(axis=1)

# Group indicator
bayes_subject_piv['group'] = has_error_df.pivot_table(index='usubject',
                                                      values='condition_indicator',
                                                      aggfunc=lambda x: np.unique(x)[0])

# Melt it back down to get a table of per-error-type count data
# including group indicators and totals
melted_df = pd.melt(bayes_subject_piv, 
                    id_vars=['group'], 
                    value_vars=[1, 2, 3], 
                    value_name='error_count')

In [ ]:
bayes_model_df.sample(n=5)

In [ ]:
# def create_coeff(name, shape):
#     sigma = pm.Gamma(f'sigma_{name}', 1.64, .32)
#     noise = pm.Normal(f'noise_{name}', mu=0, sd=1, shape=shape)
#     a = pm.Deterministic(f'a_{name}', noise * sigma)
    
#     return a


# def create_model(position_index, condition_index, subject_index, y, batch_size,
#                  num_subjects, num_position_levels, num_condition_levels): 
    
#     num_interaction_levels = num_position_levels * num_condition_levels
    
#     with pm.Model() as model:
#         b0 = pm.Normal('intercept', 0, tau=1/2**2)
        
#         a_position = create_coeff('position', num_position_levels)
#         a_condition = create_coeff('condition', num_condition_levels)
#         a_subject = create_coeff('subject', num_subjects)

#         b_position = a_position - a_position.mean()
#         b_condition = a_condition - a_condition.mean()
#         b_subject = a_subject - a_subject.mean()
        
#         mu = b0
#         mu += b_position[position_index]
#         mu += b_condition[condition_index]
#         mu += b_subject[subject_index]
#         mu = pm.Deterministic('mu', T.nnet.sigmoid(mu))
        
#         kappa = pm.Gamma('beta_variance', .01, .01)
#         alpha = mu * kappa + 1
#         beta = (1 - mu) * kappa + 1
        
#         theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=batch_size)
#         y = pm.Bernoulli('targets', p=theta, observed=y)
        
#         position_contrast = pm.Deterministic('position_contrast', 
#                                              a_position[1] - a_position[0])
#         condition_contrast = pm.Deterministic('condition_contrast', 
#                                               a_condition[1] - a_condition[0])
        
#     return model

# y = bayes_model_df['errors_2'].values
# num_obs = len(y)

# position_index = bayes_model_df.position_type.astype('category').cat.codes.values
# condition_index = bayes_model_df.condition_indicator.astype('category').cat.codes.values
# subject_index = bayes_model_df.usubject.astype('category').cat.codes.values

# num_position_levels = len(bayes_model_df.position_type.unique())
# num_condition_levels = len(bayes_model_df.condition_indicator.unique())
# num_subjects = len(bayes_model_df.usubject.unique())

# model = create_model(position_index, condition_index, subject_index, y, num_obs, 
#                      num_subjects, num_position_levels, num_condition_levels)

# with model:
#     step = pm.NUTS(target_accept=.98)
#     trace = pm.sample(4000, step, tune=500, chains=8, cores=1)

In [ ]:
per_board_pivot = bayes_model_df.pivot_table(index=('usubject', 'position_id'),
                                            values=('errors_2', 'occupied'),
                                            aggfunc=np.sum)
per_board_pivot['position_type'] = bayes_model_df.pivot_table(
    index=('usubject', 'position_id'), values='position_type', 
    aggfunc=lambda x: x.unique()[0])

per_board_pivot['condition_indicator'] = bayes_model_df.pivot_table(
    index=('usubject', 'position_id'), values='condition_indicator', 
    aggfunc=lambda x: x.unique()[0])

In [ ]:
per_board_pivot['usubject'] = per_board_pivot.index.get_level_values('usubject')
per_board_pivot['position_id'] = per_board_pivot.index.get_level_values('position_id')

per_board_pivot.reset_index(drop=True, inplace=True)

In [ ]:
per_board_pivot['interaction'] = per_board_pivot.apply(
    lambda row: str(row['position_type']) + str(row['condition_indicator']),
    axis=1
)

In [ ]:
piv_idx_ = ('usubject', 'position_type', 'condition_indicator',)
piv_vals_ = ('errors_2', 'occupied')
high_piv = bayes_model_df.pivot_table(index=piv_idx_, values=piv_vals_, aggfunc=np.sum)

In [ ]:
high_piv['usubject'] = high_piv.index.get_level_values('usubject')
high_piv['position_type'] = high_piv.index.get_level_values('position_type')
high_piv['condition_indicator'] = high_piv.index.get_level_values('condition_indicator')
high_piv.reset_index(drop=True, inplace=True)

In [ ]:
high_piv['interaction'] = high_piv.apply(
    lambda row: str(row['position_type']) + str(row['condition_indicator']),
    axis=1
)

high_piv

In [ ]:
# n = per_board_pivot['occupied'].values
# y = per_board_pivot['errors_2'].values

n = high_piv['occupied'].values
y = high_piv['errors_2'].values

num_position_levels = len(bayes_model_df.position_type.unique())
num_condition_levels = len(bayes_model_df.condition_indicator.unique())
num_interaction_levels = num_position_levels * num_condition_levels
num_subjects = len(bayes_model_df.usubject.unique())

num_obs = len(y)
batch_size = num_obs

# x_batches = pm.Minibatch(x, batch_size=batch_size)
# n_batches = pm.Minibatch(n, batch_size=batch_size)
# y_batches = pm.Minibatch(y, batch_size=batch_size)

# p2e = per_board_pivot.position_type.astype('category').cat.codes.values
# c2e = per_board_pivot.condition_indicator.astype('category').cat.codes.values
# i2e = per_board_pivot.interaction.astype('category').cat.codes.values
# s2e = per_board_pivot.usubject.astype('category').cat.codes.values

p2e = high_piv.position_type.astype('category').cat.codes.values
c2e = high_piv.condition_indicator.astype('category').cat.codes.values
i2e = high_piv.interaction.astype('category').cat.codes.values
s2e = high_piv.usubject.astype('category').cat.codes.values

In [ ]:
# No condition split-plot BANOVA design
# TODO:
# This needs to only have coefficients for subject and position type
# Then a deterministic var that computes 
# mean(subjects[trained]) - mean(subjects[untrained])

def create_coeff(name, shape):
    sigma = pm.Gamma(f'sigma_{name}', 1.64, .32)
    a = pm.Normal(f'a_{name}', mu=0, tau=1/sigma**2, shape=shape)
    return a

def create_model(position_index, condition_index, interaction_index, subject_index, 
                 y, n, batch_size,
                 num_position_levels, num_condition_levels, num_interaction_levels, 
                 num_subjects): 
    
    with pm.Model() as model:
        a0 = pm.Normal('intercept', 0, tau=1/2**2)
        a_position = create_coeff('position', num_position_levels)
        a_condition = create_coeff('condition', num_condition_levels)
        a_interaction = create_coeff('interaction', num_interaction_levels)
        
        mu = a0
        mu += a_position[position_index]
        mu += a_condition[condition_index]
        mu += a_interaction[interaction_index]
        mu = pm.Deterministic('mu', mu)

        omega = pm.Deterministic('omega', T.nnet.sigmoid((mu)))
        kappa = pm.Gamma('beta_variance', .01, .01)
        alpha = omega * (kappa - 2) + 1
        beta = (1 - omega) * (kappa - 2) + 1
        
        theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=num_subjects)
        y = pm.Binomial('targets', p=theta[subject_index], n=n, observed=y)

        a = T.concatenate([a_position, a_condition, a_interaction, a_subject])
        m = pm.Deterministic('m', a0 + a)
        bb0 = pm.Deterministic('bb0', T.mean(m))
        bb = pm.Deterministic('bb', m - bb0)
        
        position_contrast = pm.Deterministic('bb_pos', bb[1] - bb[0])
        condition_contrast = pm.Deterministic('bb_con', bb[3] - bb[2])
        
    return model

In [ ]:
# No subject split-plot BANOVA design
# In standard BANOVA, when each subject belongs to only one condition,
# condition and subject are in a sense confounded

# That is, all variability due to condition 
# could be capture in the subject coefficients

# This modification of ANOVA treats subjects as noise instead,
# with a shared logistic prior 
# that only includes condition, position type, and interaction

# I suppose one further alternative is to do some post processing on
# sampled values: for each condition coefficient, 
# add the mean of all of that condition's subjects' coefficients
# and subtract that mean from all that condition's subjects' coefficients

# This will "correct" the estimated condition coefficients 
# to contain *all* the variance from subjects within that condition

def create_coeff(name, shape):
    sigma = pm.Gamma(f'sigma_{name}', 1.64, .32)
    a = pm.Normal(f'a_{name}', mu=0, tau=1/sigma**2, shape=shape)
    return a

def create_model(position_index, condition_index, interaction_index, subject_index, 
                 y, n, batch_size,
                 num_position_levels, num_condition_levels, num_interaction_levels, 
                 num_subjects): 
    
    with pm.Model() as model:
        a0 = pm.Normal('intercept', 0, tau=1/2**2)
        a_position = create_coeff('position', num_position_levels)
        a_condition = create_coeff('condition', num_condition_levels)
        a_interaction = create_coeff('interaction', num_interaction_levels)
        
        mu = a0
        mu += a_position[position_index]
        mu += a_condition[condition_index]
        mu += a_interaction[interaction_index]
        mu = pm.Deterministic('mu', mu)

        omega = pm.Deterministic('omega', T.nnet.sigmoid((mu)))
        kappa = pm.Gamma('beta_variance', .01, .01)
        alpha = omega * (kappa - 2) + 1
        beta = (1 - omega) * (kappa - 2) + 1
        
        theta = pm.Beta('theta', alpha=alpha, beta=beta, 
                        shape=batch_size)
        y = pm.Binomial('targets', p=theta, n=n, observed=y)

        a = T.concatenate([a_position, a_condition, a_interaction])
        m = pm.Deterministic('m', a0 + a)
        bb0 = pm.Deterministic('bb0', T.mean(m))
        bb = pm.Deterministic('bb', m - bb0)
        
        position_contrast = pm.Deterministic('bb_pos', bb[1] - bb[0])
        condition_contrast = pm.Deterministic('bb_con', bb[3] - bb[2])
        
    return model

In [ ]:
def create_coeff(name, shape, sigma=None):
    if sigma is None:
#         sigma = pm.HalfStudentT(f'sigma_{name}', nu=2, lam=.1)
          sigma = pm.Gamma(f'sigma', 1.64, .32)
#     noise = pm.Normal(f'noise_{name}', mu=0, sd=1, shape=shape)
#     a = pm.Deterministic(f'a_{name}', noise * sigma)
    a = pm.Normal(f'a_{name}', mu=0, tau=1/sigma**2, shape=shape)
    return a

def create_model(position_index, condition_index, interaction_index, subject_index, 
                 y, n, batch_size,
                 num_position_levels, num_condition_levels, num_interaction_levels, 
                 num_subjects): 
    
    with pm.Model() as model:
        sigma = pm.Gamma(f'sigma', 1.64, .32)
        
        a0 = pm.Normal('intercept', 0, tau=1/2**2)
        a_position = create_coeff('position', num_position_levels)
        a_condition = create_coeff('condition', num_condition_levels)
        a_interaction = create_coeff('interaction', num_interaction_levels)
        a_subject = create_coeff('subject', num_subjects)
        
        mu = a0
        mu += a_position[position_index]
        mu += a_condition[condition_index]
        mu += a_interaction[interaction_index]
        mu += a_subject[subject_index]
        mu = pm.Deterministic('mu', mu)

        omega = pm.Deterministic('omega', pm.invlogit(mu))
        kappa = pm.Gamma('beta_variance', .1, .1)
        alpha = omega * (kappa - 2) + 1
        beta = (1 - omega) * (kappa - 2) + 1
        
        theta = pm.Beta('theta', alpha=alpha, beta=beta, 
                        shape=batch_size)
        y = pm.Binomial('targets', p=theta, n=n, observed=y)

#         a = T.concatenate([a_position, a_condition, a_interaction])
#         m = pm.Deterministic('m', a0 + a)
#         bb0 = pm.Deterministic('bb0', T.mean(m))
#         bb = pm.Deterministic('bb', m - bb0)

        # Reparameterize as deflections
        # First, correct condition by subjects
        
        # Get the mean per group of subjects
        a_sub_x_cond = a_subject.reshape((-1, 2))
        m_sub_x_cond = T.mean(a_sub_x_cond, axis=0) # shape: (1, 2)
        
        # Add it to conditions and subtract it from subjects
        b_condition_ = pm.Deterministic('b_condition_', a_condition + m_sub_x_cond)
        b_subject_ = a_sub_x_cond - m_sub_x_cond
        b_subject_ = pm.Deterministic('b_subject_', b_subject_.reshape((-1, )))
    
        m_position = T.mean(a_position)
        m_condition = T.mean(b_condition_)
        m_subject = T.mean(b_subject_)
        m_interaction = T.mean(a_interaction)

        b_position = pm.Deterministic('b_position', a_position - m_position)        
        b_condition = pm.Deterministic('b_condition', b_condition_ - m_condition)
        b_interaction = pm.Deterministic('b_interaction', a_interaction - m_interaction)
        b_subject = pm.Deterministic('b_subject', b_subject_ - m_subject)
        b0 = pm.Deterministic('b_intercept', 
                              a0 + m_position + m_condition + m_interaction + m_subject)
        
        position_contrast = pm.Deterministic('contrast_position', 
                                             b_position[1] - b_position[0])
        condition_contrast = pm.Deterministic('contrast_condition', 
                                              b_condition[1] - b_condition[0])
        
    return model

# Last edited 2019.09.14 8am

In [ ]:
class BANOVAModel(object):
    def __init__(self, batch_size):
        self.model = pm.Model()
        self.batch_size = batch_size
    
    def create_coefficient(self, name, shape, sigma=None):
        if sigma is None:
            sigma = pm.Gamma(f'sigma_{name}', 1.64, .32)

        a = pm.Normal(f'a_{name}', mu=0, tau=1/sigma**2, shape=shape)
        return a
    
    def create_likelihood(self, omega, n, y):
        kappa = pm.Gamma('beta_variance', .01, .01)
        alpha = omega * (kappa - 2) + 1
        beta = (1 - omega) * (kappa - 2) + 1
        
        theta = pm.Beta('theta', alpha=alpha, beta=beta, 
                        shape=self.batch_size)
        
        targets = pm.Binomial('targets', p=theta, n=n, observed=y)
        
        return targets
    
    def create_model(self, position_index, interaction_index, subject_index, n, y):
        with self.model:
            a_s = self.create_coefficient('subject', self.num_subjects)
            a_p = self.create_coefficient('position', self.num_position_levels)
            a_i = self.create_coefficient('interaction', self.num_interaction_levels)
            
            mu = a_position[position_index]
            mu += a_interaction[interaction_index]
            mu += a_subject[subject_index]
            mu = pm.Deterministic('mu', mu)

            omega = pm.Deterministic('omega', pm.invlogit(mu))
            
            targets = self.create_likelihood(omega, n, y)

In [ ]:
def create_coeff(name, shape, sigma=None):
    if sigma is None:
        sigma = pm.Gamma(f'sigma_{name}', 1.64, .32)
        
    a = pm.Normal(f'a_{name}', mu=0, tau=1/sigma**2, shape=shape)
    return a

def create_model(position_index, condition_index, interaction_index, subject_index, 
                 y, n, batch_size,
                 num_position_levels, num_condition_levels, num_interaction_levels, 
                 num_subjects): 
    
    # Model without explicit condition or intercept
    with pm.Model() as model: 
        # Coefficients with independent priors on variance
        a_position = create_coeff('position', num_position_levels)
        a_interaction = create_coeff('interaction', num_interaction_levels)
#         a_subject = create_coeff('subject', num_subjects)
        a_condition = create_coeff('condition', num_condition_levels)
        
        # Regression equation
        mu = a_position[position_index]
        mu += a_interaction[interaction_index]
#         mu += a_subject[subject_index]
        mu += a_condition[condition_index]
        mu = pm.Deterministic('mu', mu)

        omega = pm.Deterministic('omega', pm.invlogit(mu))
        
        # Convert to beta parameters
        kappa = pm.Gamma('beta_variance', .01, .01)
        alpha = omega * (kappa - 2) + 1
        beta = (1 - omega) * (kappa - 2) + 1
        
        # Sample probabilities and targets
        theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=batch_size)
        y = pm.Binomial('targets', p=theta, n=n, observed=y)

        # Reparameterize as deflections
        # First, correct condition by subjects
        
        # Get the mean per group of subjects
#         a_sub_x_cond = a_subject.reshape((-1, 2))
#         m_sub_x_cond = T.mean(a_sub_x_cond, axis=0) # shape: (1, 2)
        
        # Add it to conditions and subtract it from subjects
#         b_condition_ = pm.Deterministic('b_condition_', a_condition + m_sub_x_cond)
#         a_condition = pm.Deterministic('a_condition', m_sub_x_cond)
#         a_subject_ = a_sub_x_cond - m_sub_x_cond
#         a_subject_ = pm.Deterministic('a_subject_minus_condition', 
#                                       a_subject_.reshape((-1, )))
    
        m_position = T.mean(a_position)
        m_condition = T.mean(a_condition)
#         m_subject = T.mean(a_subject_)
        m_interaction = T.mean(a_interaction)
        
        b_intercept = pm.Deterministic('b_intercept', 
                                       pm.logit(m_condition + m_position + m_interaction))
        
#         b_subject = pm.Deterministic('b_subject', a_subject_ - m_subject)
        b_position = pm.Deterministic('b_position', pm.logit(a_position - m_position))   
        b_condition = pm.Deterministic('b_condition', pm.logit(a_condition - m_condition))
        b_interaction = pm.Deterministic('b_interaction', pm.logit(a_interaction - m_interaction))
        
        position_contrast = pm.Deterministic('contrast_position', 
                                             b_position[1] - b_position[0])
        condition_contrast = pm.Deterministic('contrast_condition', 
                                              b_condition[1] - b_condition[0])
        
    return model

In [ ]:
model = create_model(p2e, c2e, i2e, s2e, y, n, batch_size,
                     num_position_levels, num_condition_levels, num_interaction_levels, 
                     num_subjects)

In [ ]:
with model:
    step = pm.NUTS(target_accept=.99, max_treedepth=15)
    trace = pm.sample(4000, step, tune=6000, chains=8, cores=8)

In [ ]:
sns.set_style('white')

# pm.traceplot(trace, var_names=('b_intercept', 'b_position', 'b_condition', 'b_interaction'), )
pm.traceplot(trace, 
             var_names=('sigma_condition', 'sigma_position')
#              var_names=('sigma', )
            )
sns.despine()
plt.tight_layout();

In [ ]:
sns.set_style('white')
pm.plot_posterior(trace, 
                  var_names=('b_intercept', 'b_condition', 'b_interaction',
                             'contrast_position', 'contrast_condition'),
                  color='#87ceeb',
                  rope=(-.01, .01),
                  kind='hist',
                  ref_val=0);
sns.despine()
plt.tight_layout()

In [ ]:
# Standard split-plot BANOVA design
def create_coeff(name, shape):
    sigma = pm.Gamma(f'sigma_{name}', 1.64, .32)
#     sigma = pm.HalfSTudentT(f'sigma_{})
#     sigma = pm.HalfStudentT(f'sigma_{name}', lam=.001, nu=1,) + .1
#     noise = pm.Normal(f'noise_{name}', mu=0, sd=1, shape=shape)
#     a = pm.Deterministic(f'a_{name}', noise * (sigma + .01))
    a = pm.Normal(f'a_{name}', mu=0, tau=1/sigma**2, shape=shape)
    
    return a


def create_model(position_index, condition_index, interaction_index, subject_index, 
                 y, n, batch_size,
                 num_position_levels, num_condition_levels, num_interaction_levels, 
                 num_subjects): 
    
    with pm.Model() as model:
        a0 = pm.Normal('intercept', 0, tau=1/2**2)
        
        a_position = create_coeff('position', num_position_levels)
        a_condition = create_coeff('condition', num_condition_levels)
        a_interaction = create_coeff('interaction', num_interaction_levels)
        a_subject = create_coeff('subject', num_subjects)
        
        mu = a0
        mu += a_position[position_index]
        mu += a_condition[condition_index]
        mu += a_interaction[interaction_index]
        mu += a_subject[subject_index]
        mu = pm.Deterministic('mu', mu)

        omega = pm.Deterministic('omega', T.nnet.sigmoid((mu)))
        
        kappa = pm.Gamma('beta_variance', .01, .01)
        alpha = omega * (kappa - 2) + 1
        beta = (1 - omega) * (kappa - 2) + 1
        
        theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=batch_size)
        y = pm.Binomial('targets', p=theta, n=n, observed=y)

        a = T.concatenate([a_position, a_condition, a_interaction, a_subject])
        m = pm.Deterministic('m', a0 + a)
        bb0 = pm.Deterministic('bb0', T.mean(m))
        bb = pm.Deterministic('bb', m - bb0)
        
        position_contrast = pm.Deterministic('bb_pos', bb[1] - bb[0])
        condition_contrast = pm.Deterministic('bb_con', bb[3] - bb[2])
        
    return model

In [ ]:
model = create_model(p2e, c2e, i2e, s2e, y, n, batch_size,
                     num_position_levels, num_condition_levels, num_interaction_levels, 
                     num_subjects)

In [ ]:
with model:
    step = pm.NUTS(target_accept=.99, max_treedepth=15)
    trace = pm.sample(8000, step, tune=8000, chains=8, cores=8)

In [ ]:
sns.set_style('white')
pm.plot_posterior(trace, 
                  varnames=['bb_pos', 'bb_con'],
                  color='#87ceeb',
                  rope=(-.01, .01),
                  kind='hist',
                  ref_val=0);
sns.despine()
plt.tight_layout()

In [ ]:
intercept_dummies = pd.DataFrame(index=per_board_pivot.index, 
                                 data=np.ones(len(per_board_pivot)))
position_dummies = pd.get_dummies(per_board_pivot.position_type)
condition_dummies = pd.get_dummies(per_board_pivot.condition_indicator)
interaction_dummies = pd.get_dummies(per_board_pivot.interaction)

subject_dummies = pd.get_dummies(per_board_pivot.usubject)


In [ ]:
all_dummies = pd.concat([
    intercept_dummies, 
    position_dummies, 
    condition_dummies, 
    interaction_dummies,
    subject_dummies
], axis=1)

In [ ]:
def get_noncentered_coeffs(name, shape):
    sd = pm.HalfStudentT(f'{name}_sd', nu=4, sigma=2.5, shape=shape)
#     noise = pm.Normal(f'{name}_noise', mu=0, sd=1)
#     a = pm.Deterministic(f'{name}_a', noise * sd)
    a = pm.Normal(f'{name}_a', mu=0, sd=sd, shape=shape)
    b = a - a.mean()
    return a, b


def create_model(x, y, n, batch_size,
                 num_subjects,
                 num_position_levels, 
                 num_condition_levels): 
    
    num_interaction_levels = num_position_levels * num_condition_levels
    
    with pm.Model() as model:
        # Create intercept coefficient; no prior on variance
        b0 = pm.StudentT('intercept', mu=0, nu=2, lam=.001, shape=[1])
        a_position, b_position = get_noncentered_coeffs('position', [num_position_levels])
        a_condition, b_condition = get_noncentered_coeffs('condition', [num_condition_levels])
        _, b_interaction = get_noncentered_coeffs('interaction', [num_interaction_levels])
        _, b_subject = get_noncentered_coeffs('subject', [num_subjects])

        coeffs = T.concatenate([b0, b_position, b_condition, b_interaction, b_subject])
        coeffs = pm.Deterministic('coeffs', coeffs)

        mu = T.nnet.sigmoid(T.dot(x, coeffs))

        kappa = pm.Gamma('beta_variance', .01, .01)
        alpha = mu * kappa
        beta = (1 - mu) * kappa

        theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=batch_size)
        y = pm.Binomial('targets', n=n, p=theta, observed=y)

        # Contrasts
        position_contrast = pm.Deterministic('position_contrast', 
                                             a_position[1] - a_position[0])
        condition_contrast = pm.Deterministic('condition_contrast', 
                                              a_condition[1] - a_condition[0])
        
    return model


def create_2e_model(position_index, condition_index, subject_index, num_obs,
                    y, n,
                    num_subjects, num_position_levels, num_condition_levels):
    
    with pm.Model() as model:
        # Hyper parameters
        a_sigma = pm.Gamma('aSigma', 1.64, 0.32, shape=2)
        
        # Intercept
        a0 = pm.Normal('a0', 0.0, tau=1/2**2)
        
        # Parameters
        a_position = pm.Normal('a_position', 0.0, tau=1/a_sigma[0]**2, 
                               shape=[1, num_position_levels]) 
        a_condition = pm.Normal('a_condition', 0.0,  tau=1/a_sigma[1]**2,
                                shape=[num_condition_levels, 1])
        
        # Parameters for categories (Primary field positions)
        omega = pm.Deterministic('omega', pm.invlogit(a0 + a_position + a_condition))
        kappa = pm.Gamma('kappa', 0.01, 0.01)

        # Parameter for individual players
        theta_a = omega[position_index, condition_index] * kappa + 1
        theta_b = (1 - omega[position_index, condition_index]) * kappa + 1
        theta = pm.Beta('theta', theta_a, theta_b, shape=[num_subjects, num_obs])

        y = pm.Binomial('y', n=n, p=theta[:, subject_index], observed=y)

        # Convert a0,a to sum-to-zero b0,b 
        m = pm.Deterministic('m', a0 + a_position + a_condition)
        b0 = pm.Deterministic('b0', T.mean(m))
        b = pm.Deterministic('b', m - b0) 
    
    return model

In [ ]:
def create_nodp_model(position_index, condition_index, subject_index, 
                      y, n, batch_size,
                      num_subjects, num_position_levels, num_condition_levels): 
    
    num_interaction_levels = num_position_levels * num_condition_levels
    
    with pm.Model() as model:
        b0 = pm.Normal('intercept', 0, tau=1/2**2)
        
        sigma_position = pm.Gamma('sigma_position', 1.64, 0.32)
        noise_position = pm.Normal('noise_position', mu=0, sd=1, 
                                   shape=num_position_levels)
        a_position = pm.Deterministic('a_position', 
                                      noise_position * 1/sigma_position**2)
#         a_position = pm.Normal('a_position', mu=0, tau=1/sigma_position**2, 
#                                shape=num_position_levels)
        
        sigma_condition = pm.Gamma('sigma_condition', 1.64, 0.32)
        noise_condition = pm.Normal('noise_condition', mu=0, sd=1,
                                    shape=num_condition_levels)
        a_condition = pm.Deterministic('a_condition',
                                       noise_condition * 1/sigma_position**2)
#         a_condition = pm.Normal('a_condition', mu=0, tau=1/sigma_condition**2, 
#                                 shape=num_condition_levels)
        
        sigma_subject = pm.Gamma('sigma_subject', 1.64, 0.32)
        noise_subject = pm.Normal('noise_subject', mu=0, sd=1,
                                  shape=num_subjects)
        a_subject = pm.Deterministic('a_subject',
                                     noise_subject * 1/sigma_subject**2)
        
#         a_subject = pm.Normal('a_subject', mu=0, tau=1/sigma_subject**2,
#                               shape=num_subjects)

        b_position = a_position - a_position.mean()
        b_condition = a_condition - a_condition.mean()
        b_subject = a_subject - a_subject.mean()
        
        mu = b0
        mu += b_position[position_index]
        mu += b_condition[condition_index]
        mu += b_subject[subject_index]
        
#         mu = T.nnet.sigmoid(mu)
#         mu = pm.Deterministic('mu', T.nnet.sigmoid(mu))
        mu = pm.Deterministic('mu', pm.invlogit(mu))
        
        kappa = pm.Gamma('beta_variance', .01, .01)
        alpha = mu * kappa
        beta = (1 - mu) * kappa
        
        theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=batch_size)
        y = pm.Binomial('targets', n=n, p=theta, observed=y)
        
        position_contrast = pm.Deterministic('position_contrast', 
                                             a_position[1] - a_position[0])
        condition_contrast = pm.Deterministic('condition_contrast', 
                                              a_condition[1] - a_condition[0])
        
    return model

In [ ]:
x = all_dummies.values
n = per_board_pivot['occupied'].values
y = per_board_pivot['errors_2'].values

num_obs = x.shape[0]

# batch_size = 128
batch_size = num_obs

x_batches = pm.Minibatch(x, batch_size=batch_size)
n_batches = pm.Minibatch(n, batch_size=batch_size)
y_batches = pm.Minibatch(y, batch_size=batch_size)

num_subjects = len(subject_dummies.columns)
num_position_levels = 2
num_condition_levels = 2

# anova = create_model(x, y, n, num_obs,
#                      num_subjects, num_position_levels, num_condition_levels)
# batched_anova = create_model(x_batches, y_batches, n_batches, batch_size,
#                              num_subjects, num_position_levels, num_condition_levels)

p2e = per_board_pivot.position_type.astype('category').cat.codes.values
c2e = per_board_pivot.condition_indicator.astype('category').cat.codes.values
s2e = per_board_pivot.usubject.astype('category').cat.codes.values
# model_2e = create_2e_model(p2e, c2e, s2e, num_obs, y, n, 
#                            num_subjects, num_position_levels, num_condition_levels)
model_nodp = create_nodp_model(p2e, c2e, s2e, y, n, num_obs,
                               num_subjects, num_position_levels, num_condition_levels)

In [ ]:
with model_nodp:
    step = pm.NUTS(target_accept=.99)
    trace = pm.sample(2000, step, tune=500, cores=8, init='advi')

In [ ]:
pm.traceplot(trace);

In [ ]:
sns.set_style('white')
pm.plot_posterior(trace, 
                  varnames=['intercept', 'position_contrast', 'condition_contrast'],
                  color='#87ceeb',
                  rope=(-.01, .01),
                  kind='hist',
                  ref_val=0);
sns.despine()
plt.tight_layout()

In [ ]:
with batched_anova:
    svgd = pm.SVGD(n_particles=200)
    approximation = pm.fit(1000, method=svgd, obj_optimizer=pm.sgd(learning_rate=0.01),
                           callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
    
with anova:
    step = pm.NUTS(target_accept=.99)
    hierarchical_trace = pm.sample(2000, step, 
                                   tune=500, cores=8,
                                   start=approximation.sample()[0], 
                                   progressbar=True)
#     anova_trace = pm.sample(4000, tune=500, cores=4, init='advi', n_init=1000,
#                             nuts_kwargs={'target_accept': .99})
    

In [ ]:
approximation.sample(1000)['position_contrast']

In [ ]:
sns.kdeplot(approximation.sample(10000)['intercept'].flatten(), label='SVGD');

In [ ]:
with pm.Model() as logistic_anova:
    # Create placeholder lists for PyMC RVs
    sds = []
    coeffs = []

    # Create intercept coefficient; no prior on variance
    b0 = pm.StudentT('intercept', mu=0, nu=2, lam=.001, shape=[1])
    a_position, b_position = get_noncentered_coeffs('position', [2])
    a_condition, b_condition = get_noncentered_coeffs('condition', [2])
    _, b_interaction = get_noncentered_coeffs('interaction', [4])
    _, b_subject = get_noncentered_coeffs('subject', [len(subject_dummies.columns)])
    
    coeffs = T.concatenate([b0, b_position, b_condition, b_interaction, b_subject])
#     coeffs = T.concatenate([b0, b_position, b_condition, b_interaction])
    coeffs = pm.Deterministic('coeffs', coeffs)
    
#     mu = T.nnet.sigmoid(T.dot(all_dummies.values, coeffs))
    mu = T.nnet.sigmoid(T.dot(x_batches, coeffs))

    kappa = pm.Gamma('beta_variance', .01, .01)
    alpha = mu * kappa
    beta = (1 - mu) * kappa

#     theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=len(per_board_pivot))
    theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=batch_size)
    y = pm.Binomial('targets', 
                    n=n_batches,
                    p=theta,
                    observed=y_batches)
    
    # Contrasts
    position_contrast = pm.Deterministic('position_contrast', 
                                         a_position[1] - a_position[0])
    condition_contrast = pm.Deterministic('condition_contrast', 
                                          a_condition[1] - a_condition[0])

In [ ]:
with logistic_anova:
    svgd = pm.SVGD()
    approximation = pm.fit(10000, method=svgd, obj_optimizer=pm.sgd(learning_rate=0.01))

In [ ]:
plt.plot(approximation.hist)

In [ ]:
sns.kdeplot(approximation.sample(100000)['position_contrast'], label='SVGD');

In [ ]:
with logistic_anova:
    anova_trace = pm.sample(8000, tune=1000, cores=4,
                            nuts_kwargs={'target_accept': .99})

In [ ]:
sns.set_style('white')
pm.plot_posterior(anova_trace, 
                  varnames=['intercept', 'position_contrast', 'condition_contrast'],
                  color='#87ceeb',
                  rope=(-.01, .01),
                  kind='hist',
                  ref_val=0);
sns.despine()
plt.tight_layout()

In [ ]:
with pm.Model() as logistic_anova:
    # Create placeholder lists for PyMC RVs
    sds = []
    coeffs = []

    # Create intercept coefficient; no prior on variance
    a0 = pm.Normal('intercept', mu=0, sd=10, shape=[1])
    
    sds_position = pm.HalfStudentT('a_position_std', nu=4, sigma=2.5, shape=[2])
    a_position = pm.Normal('a_position', mu=0, sd=sds_position, shape=[2])
    
    sds_condition = pm.HalfStudentT('a_condition_std', nu=4, sigma=2.5, shape=[2])
    a_condition = pm.Normal('a_condition', mu=0, sd=sds_condition, shape=[2])
    
    sds_interaction = pm.HalfStudentT('a_interaction_std', nu=4, sigma=2.5, shape=[4])
    a_interaction = pm.Normal('a_interaction', mu=0, sd=sds_interaction, shape=[4])
    
    sds_subject = pm.HalfStudentT('a_subject_sds', nu=4, sigma=2.5, 
                                  shape=[len(subject_dummies.columns)])
    a_subject = pm.Normal('a_subject', mu=0, sd=sds_subject, 
                          shape=[len(subject_dummies.columns)])
    
    b0 = a0 #+ a_position.mean() + a_condition.mean() + a_subject.mean() + a_interaction.mean()
    b_position = a_position - a_position.mean()
    b_condition = a_condition - a_condition.mean()
    b_interaction = a_interaction - a_interaction.mean()
    b_subject = a_subject - a_condition.mean()
    
    coeffs = T.concatenate([b0, b_position, b_condition, b_interaction, b_subject])
    coeffs = pm.Deterministic('coeffs', coeffs)
    
    mu = T.nnet.sigmoid(T.dot(all_dummies.values, coeffs))

#     kappa = pm.HalfNormal('beta_variance', sigma=1)
    kappa = pm.Gamma('beta_variance', mu=1, sigma=10)
    alpha = mu * kappa
    beta = (1 - mu) * kappa

    theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=len(per_board_pivot))
    y = pm.Binomial('targets', 
                    n=per_board_pivot['occupied'].values,
                    p=theta,
                    observed=per_board_pivot['errors_2'].values)
    
    # Contrasts
    position_contrast = pm.Deterministic('position_contrast', 
                                         a_position[1] - a_position[0])
    condition_contrast = pm.Deterministic('condition_contrast', 
                                          a_condition[1] - a_condition[0])

In [ ]:
with logistic_anova:
    anova_trace = pm.sample(8000, tune=1000, cores=4,
                            nuts_kwargs={'target_accept': .99})

In [ ]:
sns.set_style('white')
pm.plot_posterior(anova_trace, 
                  varnames=['coeffs'],
                  color='#87ceeb',
                  rope=(-.01, .01),
                  kind='hist',
                  ref_val=0);
sns.despine()
plt.tight_layout()

In [ ]:
# Finally (finally!) use count df and patsy formula 
# to generate regression indicator endogenous data
# and count exogenous data

formula = 'errors_2 ~ C(usubject, Sum) + C(condition_indicator, Sum) * C(position_type, Sum)'
# formula = 'errors_2 ~ C(condition_indicator) + C(position_type)'
endogenous_df, exogenous_df = pt.dmatrices(formula, per_board_pivot, 
                                           return_type='dataframe', 
                                           NA_action='raise')

In [ ]:
with pm.Model() as logistic_anova:
    # Create placeholder lists for PyMC RVs
    sds = []
    coeffs = []

    # Create intercept coefficient; no prior on variance
    b0 = pm.Normal('intercept', mu=0, sd=1, shape=[1])
    sds = pm.HalfStudentT('coeff_std', nu=4, sigma=2.5, shape=exogenous_df.shape[1]-1)
    b = pm.Normal('coeffs', mu=0, sd=sds, shape=exogenous_df.shape[1]-1)

    # Estimate mean of beta prior as sigmoid
    mu = T.nnet.sigmoid(T.dot(exogenous_df.values, T.concatenate([b0, b])))

#     kappa = pm.HalfNormal('beta_variance', sigma=1)
    kappa = pm.Gamma('beta_variance', mu=1, sigma=10)
    alpha = mu * kappa
    beta = (1 - mu) * kappa

    theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=len(exogenous_df))
#     y = pm.Bernoulli('tagets', p=theta, observed=endogenous_df['errors_2'].values)
    y = pm.Binomial('targets', 
                    n=per_board_pivot['occupied'].values,
                    p=theta,
                    observed=endogenous_df['errors_2'].values)

In [ ]:
with logistic_anova:
    anova_trace = pm.sample(8000, tune=1000, cores=4,
                            nuts_kwargs={'target_accept': .99})

In [ ]:
sns.set_style('white')
pm.plot_posterior(anova_trace, 
                  varnames=['intercept', 'coeffs'],
                  color='#87ceeb',
                  rope=(-.01, .01),
                  ref_val=0);
sns.despine()
plt.tight_layout()

In [ ]:
exogenous_df.sample(n=5)

In [ ]:
print(list(zip(range(len(exogenous_df.columns) - 1), exogenous_df.columns.values[1:])))

In [ ]:
# Instantiate and display poisson model
poisson_model = build_poisson_model(exogenous_df, endogenous_df['error_count'].values)
pm.model_to_graphviz(poisson_model)

In [ ]:
with poisson_model:
    poisson_trace = pm.sample(16000, tune=4000, cores=4, 
                              nuts_kwargs={'target_accept': .99})

In [ ]:
sns.set(style='white')

In [ ]:
pm.traceplot(poisson_trace)
sns.despine();

In [ ]:
pm.plot_posterior(poisson_trace, 
                  varnames=[f'b_error_type[T.{i}]_group' for i in range(1, 4)],
                  color='#87ceeb',
                  rope=(-.01, .01),
                  ref_val=0);

In [ ]:
pm.summary(poisson_trace)

In [ ]:
samples = poisson_trace.get_values('b_error_type[T.2]_group')
sd = np.std(samples)

np.where((sd > samples) & (samples > -sd))